In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_regression
from sklearn.preprocessing import PolynomialFeatures, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.preprocessing import LabelEncoder
import csv
from sklearn.decomposition import PCA
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# Load the training data
df = pd.read_csv('train.csv')
df.drop_duplicates(inplace=True)
df = df.drop(columns=['sub_area'], axis=1)

# Separate features (X) and target variable (y)
X = df.loc[:, df.columns != 'price_doc']
y = df[['price_doc']]


In [ ]:
# Encode categorical variables
object_cols = X.select_dtypes(include=['object']).columns
label = LabelEncoder()
for col in object_cols:
    X[col] = label.fit_transform(X[col].astype(str))


In [ ]:
# Apply feature scaling
scaler = MinMaxScaler()
X = scaler.fit_transform(X)


In [ ]:
# Apply PCA
PCA_s = PCA(n_components=30)
X = PCA_s.fit_transform(X)


In [ ]:
# Apply variance threshold
var = VarianceThreshold(threshold=0.01)
X = var.fit_transform(X)


In [ ]:
# Feature selection
skb = SelectKBest(f_regression, k=15)
X = skb.fit_transform(X, y)


In [ ]:
# Polynomial features
poly = PolynomialFeatures(3, include_bias=False)
X = poly.fit_transform(X)


In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:

# Build a neural network model
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))


In [ ]:
# Make predictions on the test data
y_pred_test = model.predict(X_test)


In [ ]:
# Evaluate the model
mse = mean_squared_error(y_test, y_pred_test)
print(f'Mean Squared Error on Test Data: {mse}')


In [ ]:
# Load the test data
df_test = pd.read_csv('test.csv')
df_test = df_test.drop(['row ID'], axis=1)
df_test = df_test.drop(['sub_area'], axis=1)


In [ ]:
# Encode categorical variables in the test data
label_test = LabelEncoder()
for col in object_cols:
    df_test[col] = label_test.fit_transform(df_test[col].astype(str))


In [ ]:
# Apply feature scaling, PCA, variance threshold, feature selection, and polynomial features to the test data
df_test = scaler.transform(df_test)
df_test = PCA_s.transform(df_test)
df_test = var.transform(df_test)
df_test = skb.transform(df_test)
df_test = poly.transform(df_test)

In [ ]:
# Make predictions on the test data using the trained neural network
y_pred_test = model.predict(df_test)

In [ ]:

# Save predictions to a CSV file
filepath = 'prediction_25nov_neural_network.csv'
with open(filepath, mode='w', newline='') as file:
    writer = csv.writer(file)

    # Add column headers
    writer.writerow(['row ID', 'price_doc'])

    # Writing predictions with corresponding index (starting from 1)
    for c, prediction in enumerate(y_pred_test, start=1):
        writer.writerow([c, prediction[0]])